In [1]:
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import numpy as np

In [2]:
# Construct S3 key for accessing data within S3 bucket
bucket = "s3://hls-debug-output"
run_id = "brad_test_samples"
hls_granule_id = "HLS.L30.T22JBQ.2024055T133022.v2.0"
sr_key = f"{bucket}/{run_id}/{hls_granule_id}/{hls_granule_id}"
sat_id = hls_granule_id.split(".")[1]

In [3]:
# Open raster file and apply scaling and masking
def open_file(base, band):
    print(f"Reading {base}.{band}.tif")
    with rasterio.open(f"{base}.{band}.tif") as src:
        data = src.read(1, masked=True)  # Read the raster data, masking nodata values
        metadata = src.meta  # Get the metadata attributes
    return data, metadata

In [4]:
# Open dataset for each band and merge into a single dataset
def open_and_merge_files(base, bands):
    datasets = []
    for band in bands:
        data, metadata = open_file(base, band)
        datasets.append((data, metadata))
        
        # Merge the datasets
    merged_data, merged_transform = merge(datasets)
    merged_metadata = metadata.copy()
    merged_metadata.update({
        'transform': merged_transform,
        'count': len(bands)
    })

    return merged_data, merged_metadata

In [5]:
# Define Spectral Bands mapping function
def get_common_band_names(sat_id):
    sr_bands_l30 = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07']
    sr_bands_s30 = ['B02', 'B03', 'B04', 'B8A', 'B11', 'B12']
    common_bands = ["B", "G", "R", "NIR", "SWIR1", "SWIR2"]

    bands_mapping = {
        "L30": dict(zip(sr_bands_l30, common_bands)),
        "S30": dict(zip(sr_bands_s30, common_bands))
    }

    if sat_id not in bands_mapping:
        raise ValueError("Incorrect satellite id")

    return bands_mapping[sat_id]

In [6]:
# Call the function and receive the returned dictionary
sr_bands_common = get_common_band_names(sat_id)
print(sr_bands_common) if sr_bands_common is not None else print("Incorrect satellite id")

{'B02': 'B', 'B03': 'G', 'B04': 'R', 'B05': 'NIR', 'B06': 'SWIR1', 'B07': 'SWIR2'}


In [7]:
# Example usage
if sr_bands_common is not None:
    common_bands = list(sr_bands_common.keys())
    sr_data, sr_metadata = open_and_merge_files(sr_key, common_bands)
    print("Merged dataset shape:", sr_data.shape)
else:
    print("Unable to open dataset")

Reading s3://hls-debug-output/brad_test_samples/HLS.L30.T22JBQ.2024055T133022.v2.0/HLS.L30.T22JBQ.2024055T133022.v2.0.B02.tif
Reading s3://hls-debug-output/brad_test_samples/HLS.L30.T22JBQ.2024055T133022.v2.0/HLS.L30.T22JBQ.2024055T133022.v2.0.B03.tif
Reading s3://hls-debug-output/brad_test_samples/HLS.L30.T22JBQ.2024055T133022.v2.0/HLS.L30.T22JBQ.2024055T133022.v2.0.B04.tif
Reading s3://hls-debug-output/brad_test_samples/HLS.L30.T22JBQ.2024055T133022.v2.0/HLS.L30.T22JBQ.2024055T133022.v2.0.B05.tif
Reading s3://hls-debug-output/brad_test_samples/HLS.L30.T22JBQ.2024055T133022.v2.0/HLS.L30.T22JBQ.2024055T133022.v2.0.B06.tif
Reading s3://hls-debug-output/brad_test_samples/HLS.L30.T22JBQ.2024055T133022.v2.0/HLS.L30.T22JBQ.2024055T133022.v2.0.B07.tif


AttributeError: 'tuple' object has no attribute 'profile'